In [1]:
import sys
import random
from agent import Agent
from utils import Train, OtherMask, Switch, in_bounds, generate_array
import grid
from graphics import display_grid
import neural_net
import numpy as np

In [2]:
testgrid.rewards_dict

NameError: name 'testgrid' is not defined

In [3]:
init_must_move = {'train':(2,0),'agent':(2,2),'other1':(0,1),'switch':(0,0),'other1num':1}
#optimal reward: 0
init_must_push = {'train':(2,0),'agent':(4,3),'other1':(2,3),'switch':(0,0),'other1num':1}
    # the default grid setup means agent must hit switch for optimal payoff
#optimal reward: -0.5
init_must_kill = {'train':(2,0),'agent':(4,1),'other1':(3,2),'switch':(0,0),'other2':(2,4),'other1num':1,'other2num':4}
#optimal reward: -2.5
init_hit_switch = {'train':(2,0),'agent':(4,0),'other1':(0,0),'switch':(3,2),'other2':(2,4),'other1num':1,'other2num':4}
#optimal reward: -0.2

In [13]:
# MODEL BASED INVESTIGATION
testgrid = grid.Grid(5,init_pos=init_must_push)
agent = Agent()
fails = 0
trials = 10
mc_iters = 10
for i in range(trials):
    Q, policy = agent.mc_first_visit_control(testgrid.copy(), mc_iters)
    _,_, reward = agent.run_final_policy(testgrid.copy(), Q,nn_init=False,display=True)
    fails = fails + 1 if reward < -2.5 else fails

print("fails",fails/trials)

[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['T' '_' '_' '1' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '◉' '_']]
[-1.62, -3.8475, -1.62, -1.62, -1.62]
(0, 0)
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' 'T' '_' '1' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '◉' '_']]
[-1.8, -1.8, -1.8, -1.8, 0]
(0, -1)
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' 'T' '1' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '◉' '_' '_']]
[0, -2.0, 0, 0, 0]
(0, 0)
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' 'x' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '◉' '_' '_']]
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['T' '_' '_' '1' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '◉' '_']]
[-1.6200000000000003, -1.62, -1.62, -1.62, -1.62]
(-1, 0)
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_']
 ['_' 'T' '_' '1' '_']
 ['_' '_' '_' '◉' '_']
 ['_' '_' '_' '_' '_']]
[-1.8, 0, 0, 0, 0]
(-1, 0)
[['S' '_' '_' '_' '_']
 ['_' '_' '_' '1' '_']
 ['_' '_' 'T' '◉' '_']
 ['_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_

INVESTIGATION OF MC POLICY:

For just getting out of the way of the train:
At 10 iters, agent dies 0.19 of the time. At 20 iters, it decreases to 0.04. At 30, it's less than 0.02. Above 30 iterations, the MC algorithm effectively solves the environment.

For pushing another agent out of the way of the train (timing is important):
At 10,000 iterations, the MC first visit will let the agent die 0.2 of the time. At 20,000 iterations, the agent will always save the other person.

For needing to push 1 person into way of the train to save 5:
At 1000 iterations, agent will always push the first agent. At 100 iterations, the agent will push 0.98 of the time. Even at 30 iterations, the agent will push 0.7 of the time. If acting randomly, we would expect the agent to push 0.33 \* 0.25 = 0.083 of the time.

For needing to hit the switch to save 5 people:
At 100 iterations, agent always hits the switch. At 50 iterations, it fails 0.1 of the time. At 10 iterations, it fails 0.98 of the time.

NOTE: one key parameter that affects how long the MC agent takes is how many actions in a row the agent has to take to do the correct thing. For example, if the agent is 2 squares away from the switch, it takes only 3 correct actions in a row to hit the switch (it would do this 0.2^3 of the time anyways). If the agent is 4 squares away, it would only reach the correct sequence of actions 0.2^5 of the time.


In [4]:
"""
softmax = false means original policy scheme - epsilon greedy
to run new softmax version:
agent.mc_first_visit_control(testgrid.copy(), mc_iters, nn_init=True,cutoff=0.2,softmax = True)
*cutoff determines the probability below which an action is no longer considered
"""

testgrid = grid.Grid(5,init_pos=init_must_kill)
agent = Agent()
fails = 0
trials = 100
mc_iters = 20
for i in range(trials):
    Q, policy = agent.mc_first_visit_control(testgrid.copy(), mc_iters, nn_init=True,softmax = False)
    _,_, reward = agent.run_final_policy(testgrid.copy(), Q,nn_init=True,display=False)
    fails = fails + 1 if reward < -2.5 else fails

print("fails",fails/trials)

fails 0.853


INVESTIGATION OF DUAL MODEL POLICY:

For just getting out of the way of the train, even without any MC iterations, the dual model will always move out of the way of the train.

For pushing another agent out of the way of the train (risky for the agent), at 0 iterations, the dual model pushes the person twice (the MC policy alone will push once and move backwards), and saves the agent every time. **Interestingly, at 20 iterations, the model actually does worse, failing to move the agent 0.41 of the time.** At 100 iterations, this decreases to 0.26. At 300 iterations, it is still at 0.26. At 1000 iterations, it decreases further to 0.18. At 10,000 iterations, it will always save the person

For pushing 1 person into way of the train to save 5:
At 0 iterations, the agent never pushes the 1 person. At 10 iterations, the agent will push only 0.02 of the time. At 30 iterations, the agent will only push 0.34 of the time (fails = 0.66). At 100 iterations, the agent will push 0.8 of the time (fails = 0.2)

For needing to hit the switch to save 5 people:
At 0 iterations, the agent does not know to hit the switch and so allows 4 people to die 1.0 of the time. At 5 iterations, switch is hit 0.58 of the time. At 10 iterations, the agent will hit the switch 0.8 of the time. At 50 iterations, the agent always hits the switch.